<a href="https://colab.research.google.com/github/Cp557/concrete_crack_detection/blob/main/resnet50_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ResNet50 Model for Concrete Crack Detection
This script sets up and trains a ResNet50-based model for binary classification
of concrete images (cracked vs. non-cracked).

In [1]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf

import urllib.request
import zipfile
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# URL of the zip file
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0321EN-SkillsNetwork/concrete_data_week3.zip"

urllib.request.urlretrieve(url, "concrete_data_week3.zip")

with zipfile.ZipFile("concrete_data_week3.zip", 'r') as zip_ref:
    zip_ref.extractall("concrete_data_week3")

os.remove("concrete_data_week3.zip")

print("Download and extraction complete.")

Download and extraction complete.


In [3]:
# Global constants
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

In [4]:
# ImageDataGenerator for data augmentation and preprocessing
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

# Generator for training data
train_generator = data_generator.flow_from_directory(
    '/content/concrete_data_week3/concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

# Generator for val data
validation_generator = data_generator.flow_from_directory(
    '/content/concrete_data_week3/concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 10001 images belonging to 2 classes.
Found 5001 images belonging to 2 classes.


In [13]:
# Model architecture
model = Sequential([
    ResNet50(include_top=False, pooling='avg', weights='imagenet'),
    Flatten(),
    Dense(2, activation='softmax')
])

# Freeze the layers of the pre-trained model
model.layers[0].trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
# Get steps
train_samples = train_generator.samples
validation_samples = validation_generator.samples
steps_per_epoch = train_samples // batch_size_training
validation_steps = validation_samples // batch_size_validation


In [19]:
# Fit resnet50 model
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)],
    verbose=0,
)

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [20]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 2048)                │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │           4,098 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,600,008 (90.03 MB)

 Trainable params: 4,098 (16.01 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 8,198 (32.03 KB)

In [21]:
#save resnet50 model weights
model.save_weights('/content/drive/MyDrive/Coding/concrete_crack_detection/classifier_resnet_model.weights.h5')